<a href="https://colab.research.google.com/github/KenObata/TISMIR_notebooks/blob/main/TF_IDF_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Week8: This notebook is to get baseline model of TF-IDF.

Situation: English only (=multi-class) genre detection using SMOTE.
Split: StratifiedKfold.


### set up

In [1]:
!pip3 install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from collections import Counter

from skmultilearn.model_selection import IterativeStratification   
from sklearn.model_selection import StratifiedKFold
from scipy.sparse import csr_matrix

### Data Preparation(Kfold split)

Create dataframe for Gensim

In [27]:
import pandas as pd

DIR = '/content/drive/MyDrive/music4all/'
df_genre_by_lang = pd.read_csv(DIR + 'kaggle/df_kaggle.csv')
df_genre_by_lang = df_genre_by_lang.drop(columns = ['Unnamed: 0'])
df_genre_by_lang

,Genre,Lyrics
0,Hip-Hop,Most folks spend their days daydreaming of fin...
1,Indie,Take your cold hands and put them on my face\n...
2,Metal,Are you ready it's time for war\nWe'll break d...
3,Pop,You ask me why I change the color of my hair\n...
4,Hip-Hop,Do you believe in magic in a young girl's hear...
...,...,...
258117,R&B,I'm the best friend he's got I'd give him the ...
258118,Pop,"Bad Boys Blue ""I Totally Miss You"" I did you w..."
258119,Pop,Forgive me for the things That I never said to...
258120,Indie,The day they found a cure for AIDS The day the...


In [28]:
def load_data(df_col, y):
    texts, labels = [], []
    
    for line in df_col:
        # texts are already tokenized, just split on space
        # in a real use-case we would put more effort in preprocessing
        texts.append(line.split(' '))
    return pd.DataFrame({'texts': texts, 'labels': y})

In [29]:
data = load_data(df_genre_by_lang["Lyrics"], df_genre_by_lang["Genre"])

In [30]:
data

,texts,labels
0,"[Most, folks, spend, their, days, daydreaming,...",Hip-Hop
1,"[Take, your, cold, hands, and, put, them, on, ...",Indie
2,"[Are, you, ready, it's, time, for, war\nWe'll,...",Metal
3,"[You, ask, me, why, I, change, the, color, of,...",Pop
4,"[Do, you, believe, in, magic, in, a, young, gi...",Hip-Hop
...,...,...
258117,"[I'm, the, best, friend, he's, got, I'd, give,...",R&B
258118,"[Bad, Boys, Blue, ""I, Totally, Miss, You"", I, ...",Pop
258119,"[Forgive, me, for, the, things, That, I, never...",Pop
258120,"[The, day, they, found, a, cure, for, AIDS, Th...",Indie


In [31]:
data['labels'] = data['labels'].astype('category')
label_mapping = data['labels'].cat.categories
data['labels'] = data['labels'].cat.codes
X = data['texts']
y = data['labels']

In [32]:
type(X)

pandas.core.series.Series

In [33]:
def StratifiedKFold_feature_and_df_glove(df, feature_list, y_name):
  skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1209)  # 20% for test set 
  y = df[y_name]
  skf.get_n_splits(df[ feature_list ], y)

  splits = []

  for train_index, test_index in skf.split(df[ feature_list ], y):
      print("TRAIN:", train_index, "TEST:", test_index)
      X_train, X_test = df[ feature_list ].loc[train_index], df[ feature_list ].loc[test_index]
      y_train, y_test = y.loc[train_index], y.loc[test_index]
      splits.append({'X_train': X_train, 'X_test': X_test, 'y_train':y_train, 'y_test':y_test })
  return splits

In [34]:
def StratifiedKFold_feature_and_df(X, y):
  skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1209)  # 20% for test set 
  #y = df[y_name]
  skf.get_n_splits(X, y)#df[ feature_list ]

  splits = []

  for train_index, test_index in skf.split(X, y):#df[ feature_list ]
      print("TRAIN:", train_index, "TEST:", test_index)
      X_train, X_test = X.loc[train_index], X.loc[test_index]
      y_train, y_test = y.loc[train_index], y.loc[test_index]
      splits.append({'X_train': X_train, 'X_test': X_test, 'y_train':y_train, 'y_test':y_test })
  return splits

In [35]:
#feature_list = ["texts"] #this is BOW and TF-IDF
#splits = StratifiedKFold_feature_and_df( data, feature_list, 'labels')
splits = StratifiedKFold_feature_and_df( X,y)

TRAIN: [     1      2      4 ... 258118 258120 258121] TEST: [     0      3      6 ... 258105 258112 258119]
TRAIN: [     0      1      2 ... 258119 258120 258121] TEST: [     7     15     20 ... 258110 258113 258114]
TRAIN: [     0      1      3 ... 258118 258119 258121] TEST: [     2      5     12 ... 258091 258116 258120]
TRAIN: [     0      2      3 ... 258119 258120 258121] TEST: [     1      8     10 ... 258102 258106 258118]
TRAIN: [     0      1      2 ... 258118 258119 258120] TEST: [     4     13     16 ... 258115 258117 258121]


In [36]:
len(splits)

5

In [37]:
split0=splits[0]
print(split0['X_train'].shape)
print(split0['y_train'].shape)
print(split0['X_test'].shape)
print(split0['y_test'].shape)

(206497,)
(206497,)
(51625,)
(51625,)


In [38]:
split0['X_train']

1         [Take, your, cold, hands, and, put, them, on, ...
2         [Are, you, ready, it's, time, for, war\nWe'll,...
4         [Do, you, believe, in, magic, in, a, young, gi...
5         [People, starin', at, me, as, they, wheel, me,...
7         [Tell, Mel, Shawn, to, come, in\nWord,, yo,, o...
                                ...                        
258116    [It's, complicated, It, always, is, That's, ju...
258117    [I'm, the, best, friend, he's, got, I'd, give,...
258118    [Bad, Boys, Blue, "I, Totally, Miss, You", I, ...
258120    [The, day, they, found, a, cure, for, AIDS, Th...
258121    [Fourth, of, July, has, come,, it's, custom, t...
Name: texts, Length: 206497, dtype: object

In [39]:
split0['y_train']

1         4
2         6
4         3
5         0
7         3
         ..
258116    1
258117    8
258118    7
258120    4
258121    7
Name: labels, Length: 206497, dtype: int8

In [40]:
counter_origin = Counter(df_genre_by_lang['Genre'])
counter_origin = sorted(counter_origin.items(), key= lambda k:k[1])
counter_origin

[('Electronic', 2664),
 ('Country', 2700),
 ('Hip-Hop', 3198),
 ('R&B', 3273),
 ('Indie', 7750),
 ('Folk', 8660),
 ('Jazz', 13974),
 ('Metal', 19943),
 ('Pop', 87405),
 ('Rock', 108555)]

In [41]:
counter_code = Counter(split0['y_train'])
counter_code= sorted(counter_code.items(), key= lambda k:k[1])
counter_code

[(1, 2131),
 (0, 2160),
 (3, 2558),
 (8, 2618),
 (4, 6200),
 (2, 6928),
 (5, 11180),
 (6, 15954),
 (7, 69924),
 (9, 86844)]

In [42]:
code2genre = {
    0: 'Country'
    , 1: 'Electronic'
    , 3:'Hip-Hop'
    , 8:'R&B'
    , 4: 'Indie'
    ,2: 'Folk'
    ,5: 'Jazz'
    ,6: 'Metal'
    ,7: 'Pop'
    ,9: 'Rock'
}
code2genre

{0: 'Country',
 1: 'Electronic',
 3: 'Hip-Hop',
 8: 'R&B',
 4: 'Indie',
 2: 'Folk',
 5: 'Jazz',
 6: 'Metal',
 7: 'Pop',
 9: 'Rock'}

###  Train and Test flow

Step1: Kfold Stratified Split

Step2: For X_train, get Bag-of-Words feature using TF-IDF.

Step3: clf.fit(X_train, y_train)

Step4: transform X_test['lyric'] to TF-IDF

Step5: Predict by, clf_predict = clf.predict(X_test_tf-idf)

Step6: Compare (clf_predict, y_test)

Multinomial Nayve Bayes trained on TF-IDF embedding. Test KFold mean

### Testing

In [19]:
split0['X_train']

1         [Take, your, cold, hands, and, put, them, on, ...
2         [Are, you, ready, it's, time, for, war\nWe'll,...
4         [Do, you, believe, in, magic, in, a, young, gi...
5         [People, starin', at, me, as, they, wheel, me,...
7         [Tell, Mel, Shawn, to, come, in\nWord,, yo,, o...
                                ...                        
258116    [It's, complicated, It, always, is, That's, ju...
258117    [I'm, the, best, friend, he's, got, I'd, give,...
258118    [Bad, Boys, Blue, "I, Totally, Miss, You", I, ...
258120    [The, day, they, found, a, cure, for, AIDS, Th...
258121    [Fourth, of, July, has, come,, it's, custom, t...
Name: texts, Length: 206497, dtype: object

In [20]:
def get_needs_one_epoch(y_train):
  needs_per_genre = {}

  for genre in Counter(split0['y_train']):
    needs_per_genre[genre] = int(Counter(split0['y_train'])[genre]/10)
  #print(Counter(split0['y_train']))
  #print(needs_per_genre)
  return needs_per_genre

In [40]:
EPOCH=10
ONE_SIZE = int(len(split0['y_train'])/EPOCH)
ONE_SIZE

20649

In [22]:
needs_per_genre = get_needs_one_epoch(split0['y_train'])
needs_per_genre

{4: 620,
 6: 1595,
 3: 255,
 0: 216,
 7: 6992,
 5: 1118,
 9: 8684,
 1: 213,
 2: 692,
 8: 261}

In [ ]:

for count in range(EPOCH):
  print(Counter(split0['y_train'][ONE_SIZE*count:ONE_SIZE*(count+1)]))

Counter({9: 11243, 7: 3162, 6: 2556, 3: 763, 0: 638, 1: 530, 5: 521, 4: 435, 8: 413, 2: 388})
Counter({9: 13684, 7: 3631, 6: 2551, 2: 555, 4: 208, 1: 20})
Counter({7: 10811, 9: 8522, 6: 942, 4: 373, 8: 1})
Counter({7: 17879, 9: 2411, 2: 198, 4: 153, 8: 8})
Counter({7: 8708, 9: 7551, 5: 1783, 6: 1000, 2: 813, 4: 658, 8: 94, 1: 42})
Counter({9: 10283, 7: 4488, 5: 2038, 6: 1778, 2: 949, 4: 837, 8: 162, 1: 114})
Counter({9: 9359, 7: 5388, 5: 2126, 6: 1628, 2: 1109, 4: 802, 8: 188, 1: 49})
Counter({9: 9556, 7: 6171, 6: 1610, 5: 1441, 2: 779, 4: 672, 8: 380, 1: 40})
Counter({9: 9335, 7: 5417, 5: 1928, 6: 1889, 2: 954, 4: 822, 8: 229, 1: 75})
Counter({9: 4900, 7: 4266, 6: 2000, 3: 1795, 0: 1522, 5: 1343, 1: 1260, 4: 1238, 2: 1183, 8: 1142})


In [59]:
from nltk.corpus import words
from sklearn.metrics import accuracy_score,classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from imblearn.over_sampling import SMOTE
from sklearn.metrics import balanced_accuracy_score


split0 = splits[0]
for i, token_list in split0['X_train'].items():
  """
  document = ""
  for word in token_list:
    if word in words.words():
      document += ' '
      document += word
  """
  split0['X_train'][i] =  ' '.join(token_list)

for i, token_list in split0['X_test'].items():
  """
  document = ""
  for word in token_list:
    if word in words.words():
      document += ' '
      document += word
  """
  split0['X_test'][i] =  ' '.join(token_list)

tfidf_vectorizer = TfidfVectorizer(analyzer="word", stop_words='english', max_df=0.9, min_df=0.01)#max_features=10000
X_train_tfidf = tfidf_vectorizer.fit_transform(split0['X_train']).toarray()
X_test_tfidf = tfidf_vectorizer.transform(split0['X_test']).toarray()

df_X_train_tfidf = pd.DataFrame(X_train_tfidf
                              ,index = split0['X_train'].index
                            , columns=tfidf_vectorizer.get_feature_names())
del(X_train_tfidf)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [60]:
df_X_train_tfidf

,2x,act,afraid,age,ago,ah,ahead,ain,aint,air,...,ya,yea,yeah,year,years,yes,yesterday,yo,york,young
1,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
4,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,...,0.0,0.0,0.052036,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.078477
5,0.0,0.0,0.086976,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,...,0.0,0.0,0.000000,0.000000,0.000000,0.072762,0.0,0.000000,0.000000,0.000000
7,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.034518,0.0,0.05093,...,0.0,0.0,0.031914,0.058090,0.049161,0.000000,0.0,0.421158,0.068648,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258116,0.0,0.0,0.000000,0.0,0.0,0.268642,0.0,0.000000,0.0,0.00000,...,0.0,0.0,0.154997,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
258117,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
258118,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
258120,0.0,0.0,0.097178,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,...,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


In [23]:
DIR

'/content/drive/MyDrive/music4all/'

In [27]:
#np.savetxt(DIR +"TFIDF_kaggle_split0_epoch1.csv", X_train_tfidf[ONE_SIZE*0:ONE_SIZE*1], delimiter=",")

In [25]:
type(X_train_tfidf)

numpy.ndarray

In [26]:
X_train_tfidf[ONE_SIZE*0:ONE_SIZE*1].shape

(20649, 10000)

In [ ]:
#organize X and y into completely 1/epoch scale to do incremental fit() by epoch times.
for X_row, y_label in zip(df_X_train_tfidf, split0['y_train']):

In [63]:
X_resampled, y_resampled = SMOTE().fit_resample(df_X_train_tfidf.to_numpy(), split0['y_train']) 
print(sorted(Counter(y_resampled).items()))

[(0, 86844), (1, 86844), (2, 86844), (3, 86844), (4, 86844), (5, 86844), (6, 86844), (7, 86844), (8, 86844), (9, 86844)]


In [64]:
clf_mnb_SMOTE = MultinomialNB().fit(X_resampled, y_resampled)

predicted_mnb_SMOTE = clf_mnb_SMOTE.predict(X_test_tfidf)
print(classification_report(split0['y_test'], predicted_mnb_SMOTE) )
print(balanced_accuracy_score(split0['y_test'], predicted_mnb_SMOTE))

              precision    recall  f1-score   support

           0       0.04      0.36      0.08       540
           1       0.02      0.19      0.04       533
           2       0.11      0.35      0.17      1732
           3       0.09      0.69      0.16       640
           4       0.07      0.23      0.11      1550
           5       0.30      0.52      0.38      2794
           6       0.33      0.64      0.44      3989
           7       0.61      0.14      0.23     17481
           8       0.03      0.38      0.06       655
           9       0.65      0.11      0.18     21711

    accuracy                           0.21     51625
   macro avg       0.23      0.36      0.18     51625
weighted avg       0.53      0.21      0.22     51625

0.36123500697355354


### KFold test

In [ ]:
del(split0)
del(df_X_train_tfidf)
del(X_resampled)
del(y_resampled)

In [20]:
from nltk.corpus import words
from sklearn.metrics import accuracy_score,classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from imblearn.over_sampling import SMOTE
from sklearn.metrics import balanced_accuracy_score

for i in range(0,5):
  split0 = splits[i]
  for i, token_list in split0['X_train'].items():
    split0['X_train'][i] =  ' '.join(token_list)

  for i, token_list in split0['X_test'].items():
    split0['X_test'][i] =  ' '.join(token_list)

  tfidf_vectorizer = TfidfVectorizer(analyzer="word", stop_words='english', max_df=0.9, min_df=0.01)#max_features=10000
  X_train_tfidf = tfidf_vectorizer.fit_transform(split0['X_train']).toarray()
  X_test_tfidf = tfidf_vectorizer.transform(split0['X_test']).toarray()

  df_X_train_tfidf = pd.DataFrame(X_train_tfidf
                                ,index = split0['X_train'].index
                              , columns=tfidf_vectorizer.get_feature_names())
  del(X_train_tfidf)

  X_resampled, y_resampled = SMOTE().fit_resample(df_X_train_tfidf.to_numpy(), split0['y_train']) 
  print(sorted(Counter(y_resampled).items()))

  clf_mnb_SMOTE = MultinomialNB().fit(X_resampled, y_resampled)

  predicted_mnb_SMOTE = clf_mnb_SMOTE.predict(X_test_tfidf)
  print(classification_report(split0['y_test'], predicted_mnb_SMOTE) )
  print(balanced_accuracy_score(split0['y_test'], predicted_mnb_SMOTE))

  with open(DIR+ "TFIDF_mnb_kaggle_log.txt", "a") as f:
    print("=========split: " , i, "=================", file=f)
    print(classification_report(split0['y_test'], predicted_mnb_SMOTE) , file = f)
    print(balanced_accuracy_score(split0['y_test'], predicted_mnb_SMOTE) , file = f)


  #---------------------------------
  del(split0)
  del(df_X_train_tfidf)
  del(X_resampled)
  del(y_resampled)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[(0, 86844), (1, 86844), (2, 86844), (3, 86844), (4, 86844), (5, 86844), (6, 86844), (7, 86844), (8, 86844), (9, 86844)]
              precision    recall  f1-score   support

           0       0.04      0.35      0.08       540
           1       0.02      0.20      0.04       533
           2       0.11      0.35      0.17      1732
           3       0.09      0.69      0.16       640
           4       0.07      0.23      0.11      1550
           5       0.30      0.52      0.38      2794
           6       0.33      0.65      0.44      3989
           7       0.60      0.14      0.22     17481
           8       0.03      0.37      0.06       655
           9       0.64      0.10      0.18     21711

    accuracy                           0.21     51625
   macro avg       0.22      0.36      0.18     51625
weighted avg       0.53      0.21      0.22     51625

0.36059214466137496


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[(0, 86844), (1, 86844), (2, 86844), (3, 86844), (4, 86844), (5, 86844), (6, 86844), (7, 86844), (8, 86844), (9, 86844)]
              precision    recall  f1-score   support

           0       0.04      0.32      0.07       540
           1       0.02      0.19      0.04       533
           2       0.11      0.35      0.17      1732
           3       0.09      0.68      0.15       640
           4       0.07      0.23      0.11      1550
           5       0.29      0.54      0.38      2795
           6       0.33      0.64      0.44      3988
           7       0.61      0.14      0.23     17481
           8       0.03      0.35      0.06       655
           9       0.64      0.11      0.18     21711

    accuracy                           0.21     51625
   macro avg       0.22      0.36      0.18     51625
weighted avg       0.52      0.21      0.22     51625

0.35513757111093797


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[(0, 86844), (1, 86844), (2, 86844), (3, 86844), (4, 86844), (5, 86844), (6, 86844), (7, 86844), (8, 86844), (9, 86844)]
              precision    recall  f1-score   support

           0       0.04      0.34      0.07       540
           1       0.02      0.17      0.04       532
           2       0.11      0.36      0.17      1732
           3       0.09      0.72      0.16       640
           4       0.07      0.22      0.11      1550
           5       0.30      0.54      0.38      2795
           6       0.33      0.62      0.43      3988
           7       0.61      0.14      0.23     17481
           8       0.03      0.37      0.06       655
           9       0.63      0.10      0.18     21711

    accuracy                           0.21     51624
   macro avg       0.22      0.36      0.18     51624
weighted avg       0.52      0.21      0.22     51624

0.35944450512057063


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[(0, 86844), (1, 86844), (2, 86844), (3, 86844), (4, 86844), (5, 86844), (6, 86844), (7, 86844), (8, 86844), (9, 86844)]
              precision    recall  f1-score   support

           0       0.04      0.32      0.07       540
           1       0.02      0.18      0.04       533
           2       0.11      0.37      0.17      1732
           3       0.09      0.70      0.16       639
           4       0.07      0.21      0.10      1550
           5       0.31      0.54      0.39      2795
           6       0.33      0.63      0.43      3989
           7       0.62      0.15      0.24     17481
           8       0.04      0.43      0.07       654
           9       0.63      0.10      0.18     21711

    accuracy                           0.21     51624
   macro avg       0.23      0.36      0.19     51624
weighted avg       0.52      0.21      0.22     51624

0.3631816947550563


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[(0, 86844), (1, 86844), (2, 86844), (3, 86844), (4, 86844), (5, 86844), (6, 86844), (7, 86844), (8, 86844), (9, 86844)]
              precision    recall  f1-score   support

           0       0.04      0.34      0.07       540
           1       0.02      0.17      0.04       533
           2       0.11      0.37      0.17      1732
           3       0.09      0.68      0.15       639
           4       0.07      0.22      0.10      1550
           5       0.29      0.52      0.37      2795
           6       0.33      0.62      0.43      3989
           7       0.61      0.14      0.23     17481
           8       0.03      0.37      0.06       654
           9       0.63      0.10      0.18     21711

    accuracy                           0.20     51624
   macro avg       0.22      0.35      0.18     51624
weighted avg       0.52      0.20      0.22     51624

0.35321011511894423


take mean

In [21]:
balanced_acc = []
balanced_acc.append(0.36059214466137496)
balanced_acc.append(0.35513757111093797)
balanced_acc.append(0.35944450512057063)
balanced_acc.append(0.3631816947550563)
balanced_acc.append(0.35321011511894423)

print(sum(balanced_acc)/5 )

0.3583132061533768


### Save models

In [22]:
DIR

'/content/drive/MyDrive/music4all/'

In [23]:
import pickle
filename = DIR + 'TFIDF_MNB_kaggle_split4.sav'
pickle.dump(clf_mnb_SMOTE, open(filename, 'wb'))

Check if loading a classifier is working

In [24]:
loaded_model = pickle.load(open(filename, 'rb'))

Save the first fold clf

In [45]:
split0 = splits[0]
for i, token_list in split0['X_train'].items():
  split0['X_train'][i] =  ' '.join(token_list)

for i, token_list in split0['X_test'].items():
  split0['X_test'][i] =  ' '.join(token_list)

tfidf_vectorizer = TfidfVectorizer(analyzer="word", stop_words='english', max_df=0.9, min_df=0.01)#max_features=10000
X_train_tfidf = tfidf_vectorizer.fit_transform(split0['X_train']).toarray()
X_test_tfidf = tfidf_vectorizer.transform(split0['X_test']).toarray()

df_X_train_tfidf = pd.DataFrame(X_train_tfidf
                              ,index = split0['X_train'].index
                            , columns=tfidf_vectorizer.get_feature_names())
del(X_train_tfidf)

X_resampled, y_resampled = SMOTE().fit_resample(df_X_train_tfidf.to_numpy(), split0['y_train']) 
print(sorted(Counter(y_resampled).items()))

clf_mnb_SMOTE = MultinomialNB().fit(X_resampled, y_resampled)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[(0, 86844), (1, 86844), (2, 86844), (3, 86844), (4, 86844), (5, 86844), (6, 86844), (7, 86844), (8, 86844), (9, 86844)]


In [46]:
import pickle
filename = DIR + 'TFIDF_MNB_kaggle_split0.sav'
pickle.dump(clf_mnb_SMOTE, open(filename, 'wb'))